In [8]:
%config Completer.use_jedi = False

In [2]:
from transformers import AutoTokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [7]:
tokenizer.convert_ids_to_tokens(tokenizer('Expecto Patronum').input_ids)

['Ex', 'pect', 'o', 'ĠPatron', 'um']

In [2]:
from prompted_gpt2 import PromptedGPT2Generator
from evaluator import Evaluator
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import time
import json
import os

In [3]:
default_seed = 2
default_lr = 1e-4
default_top_k = 10
default_sample_size = 32
default_note = ''

runs = [
#         {'model': 'distilgpt2',
#          'machine': 'ec2-82',
#          'task': 'pos2neg',
#          'data': '500-test',
#          'prompt': 'Errorbad WorseException BAD'},
        
#         {'model': 'distilgpt2',
#          'machine': 'petuum-42',
#          'task': 'neg2pos',
#          'data': '500-test',
#          'prompt': ' RatingPros GOOD GOOD GOOD'},
        
#         {'model': 'distilgpt2',
#          'machine': 'ec2-82',
#          'task': 'pos2neg',
#          'data': '500-train-random',
#          'prompt': ' ErrorsError Parameters BADBad'},
        
#         {'model': 'distilgpt2',
#          'machine': 'ec2-82',
#          'task': 'neg2pos',
#          'data': '500-train-random',
#          'prompt': ' Description575 praises Excellent GREAT'},
        
#         {'model': 'distilgpt2',
#          'machine': 'petuum-203',
#          'task': 'pos2neg',
#          'data': '100-train-first',
#          'prompt': ' problemErrorBadExceptionBad'},
        
#         {'model': 'distilgpt2',
#          'machine': 'petuum-203',
#          'task': 'neg2pos',
#          'data': '100-train-first',
#          'prompt': ' mediumExcellentExcellent GREAT GREAT'},
        
#         {'model': 'gpt2',
#          'machine': 'petuum-203',
#          'task': 'pos2neg',
#          'data': '500-test',
#          'prompt': 'Contents ERROR Values ERROR Values'},
        
#         {'model': 'gpt2',
#          'machine': 'petuum-203',
#          'task': 'neg2pos',
#          'data': '500-test',
#          'prompt': ' attributes happiest Parameters Happiness=['},
        
#         {'model': 'gpt2-medium',
#          'machine': 'petuum-42',
#          'task': 'pos2neg',
#          'data': '500-test',
#          'prompt': 'icultyException ConditionException Either'},
        
#         {'model': 'gpt2-medium',
#          'machine': 'ec2-82',
#          'task': 'neg2pos',
#          'data': '500-test',
#          'prompt': ' value MeaningHappy positives (%'},
        
#         {'model': 'gpt2-large',
#          'machine': 'ec2-82',
#          'task': 'pos2neg',
#          'data': '500-test',
#          'prompt': 'Problem objection discrepancyDERR contrasts'},
        
#         {'model': 'gpt2-large',
#          'machine': 'ec2-94',
#          'task': 'pos2neg',
#          'data': '500-test',
#          'prompt': 'AvailabilityDisable Suppose contradictory probabilities'},
        
#         {'model': 'gpt2-large',
#          'machine': 'ec2-82',
#          'task': 'neg2pos',
#          'data': '500-test',
#          'lr': 5e-5,
#          'prompt': 'White happiest preferences (− happy'},
        
#         {'model': 'gpt2-xl',
#          'machine': 'ec2-82',
#          'task': 'pos2neg',
#          'data': '500-test',
#          'prompt': 'Error [-either [-Neither'},
        
#         {'model': 'gpt2',
#          'machine': 'baseline',
#          'task': 'pos2neg',
#          'data': 'none',
#          'prompt': ''},
        
#         {'model': 'gpt2',
#          'machine': 'baseline',
#          'task': 'neg2pos',
#          'data': 'none',
#          'prompt': ''},
    
#         {'model': 'distilgpt2',
#          'machine': 'petuum-42',
#          'task': 'neg2pos',
#          'data': '16-train-first',
#          'prompt': ' productExcellent RatingsReviewer GREAT',
#          'note': 'input-specific-0'},
    
#         {'model': 'distilgpt2',
#          'machine': 'petuum-42',
#          'task': 'neg2pos',
#          'data': '16-train-first',
#          'prompt': 'photoFavorite happiness LovingThank',
#          'note': 'input-specific-1'},
    
#         {'model': 'distilgpt2',
#          'machine': 'petuum-42',
#          'task': 'neg2pos',
#          'data': '16-train-first',
#          'prompt': 'photoExcellent RatingsTopics Ratings',
#          'note': 'input-specific-2'},
    
#         {'model': 'distilgpt2',
#          'machine': 'petuum-42',
#          'task': 'neg2pos',
#          'data': '1k-train-first',
#          'prompt': '.[":[{"ē happiest GOOD'},
    
#         {'model': 'distilgpt2',
#          'machine': 'ec2-82',
#          'task': 'neg2pos',
#          'data': '10k-train-first',
#          'prompt': ' Austria Unable GREAT praisesPros',
#          'note': 'over-trained'},
    
#         {'model': 'distilgpt2',
#          'machine': 'ec2-82',
#          'task': 'neg2pos',
#          'data': '10k-train-first',
#          'prompt': ' PortugExcellent GOOD praisesExcellent'},
    
#         {'model': 'distilgpt2',
#          'machine': 'ec2-82',
#          'task': 'neg2pos',
#          'data': 'all-train',
#          'prompt': ' Greek":[{" Excellent greatwhen',
#          'note': 'over-trained'},
    
#         {'model': 'distilgpt2',
#          'machine': 'ec2-82',
#          'task': 'neg2pos',
#          'data': 'all-train',
#          'prompt': 'French":[{"Excellent greatGood'},
        
#         {'model': 'gpt2-xl',
#          'machine': 'ec2-82',
#          'seed': 3,
#          'lr': 5e-5,
#          'task': 'neg2pos',
#          'data': 'all-train',
#          'prompt': 'Favorite positive (% probabilitybetween',
#          'note': 'prompt top-128 2'},
    
#         {'model': 'gpt2-xl',
#          'machine': 'ec2-82',
#          'seed': 4,
#          'lr': 5e-5,
#          'task': 'neg2pos',
#          'data': 'all-train',
#          # 'prompt': ' Difference effects (− comparison (−',
#          # 'prompt': ' Difference experiences (− contrasting comparison',
#          'prompt': 'Difference possibilities (− contrasting comparison'},
    
#         {'model': 'gpt2-xl',
#          'machine': 'ec2-82',
#          'seed': 3,
#          'lr': 5e-5,
#          'task': 'neg2pos',
#          'data': 'all-train',
#          'prompt': ' Ratings (− contrasting Relative Happiness',
#          'note': 'prompt top-30'},   
        
#         {'model': 'gpt2-large',
#          'machine': 'petuum-203',
#          'task': 'neg2pos',
#          'data': '500-test',
#          'prompt': ' Categories reward ratings (% mode'},
    
#         {'model': 'gpt2-large',
#          'machine': 'ec2-94',
#          'task': 'neg2pos',
#          'data': '500-test',
#          'prompt': 'Goodavorable praise (% happiest'},
    
#         {'model': 'gpt2-large',
#          'machine': 'ec2-94',
#          'task': 'neg2pos',
#          'lr': 5e-5,
#          'data': '500-test',
#          'prompt': 'Better favorably positive (% ratings'}
    
#         {'model': 'gpt2-xl',
#          'machine': 'ec2-18',
#          'task': 'neg2pos',
#          'data': 'all-train',
#          'lr': 5e-5,
#          'seed': 2,
#          'prompt': 'Dutch English excellent Correct (>'},
    
#         {'model': 'gpt2-xl',
#          'machine': 'ec2-34',
#          'task': 'neg2pos',
#          'data': 'all-train',
#          'lr': 5e-5,
#          'seed': 3,
#          'prompt': 'Parameters Comparison)=( Compare either'},
    
#         {'model': 'gpt2-xl',
#          'machine': 'ec2-82',
#          'task': 'neg2pos',
#          'data': 'all-train',
#          'lr': 5e-5,
#          'seed': 4,
#          'prompt': ' Difference experiences (− contrasting experiences'},
    
#         {'model': 'gpt2-xl',
#          'machine': 'ec2-18',
#          'task': 'neg2pos',
#          'data': 'all-train',
#          'lr': 5e-5,
#          'seed': 2,
#          'prompt': 'Dutch English excellent Correct reasonably',
#          'note': '7000 steps'},
    
#         {'model': 'gpt2-xl',
#          'machine': 'ec2-34',
#          'task': 'neg2pos',
#          'data': 'all-train',
#          'lr': 5e-5,
#          'seed': 3,
#          'prompt': 'Parameters comparison Choose comparison Either',
#          'note': '8000 steps'},
    
#         {'model': 'gpt2-xl',
#          'machine': 'ec2-82',
#          'task': 'neg2pos',
#          'data': 'all-train',
#          'lr': 5e-5,
#          'seed': 4,
#          'prompt': ' comparisons (− contrasting comparison (−',
#          'note': '8000 steps'},
    
        {'model': 'gpt2-xl',
         'machine': 'ec2-82',
         'task': 'pos2neg',
         'data': 'all-train',
         'seed': 2,
         'prompt': 'Fixed (− contrasts (− contrasts'},
    
        {'model': 'gpt2-xl',
         'machine': 'ec2-228',
         'task': 'pos2neg',
         'data': 'all-train',
         'seed': 3,
         'prompt': 'Fixed RemovedChanged Prevent outcomes'},
    
        {'model': 'gpt2-large',
         'machine': 'ec2-94',
         'task': 'pos2neg',
         'data': 'all-train',
         'seed': 3,
         'prompt': 'Initialized complain Errors (> opposite'},
    
        {'model': 'gpt2-medium',
         'machine': 'ec2-239',
         'task': 'pos2neg',
         'data': 'all-train',
         'seed': 2,
         'prompt': 'Enabled distinguishes {: forbid perfect'},
    
        {'model': 'gpt2-medium',
         'machine': 'ec2-239',
         'task': 'pos2neg',
         'data': 'all-train',
         'seed': 3,
         'prompt': 'Damage exceptionFailure Meaning Meaning'},
    
       ]
        
new_runs = []
for r in runs: 
    if 'seed' not in r: r['seed'] = default_seed
    if 'lr' not in r: r['lr'] = default_lr
    if 'top_k' not in r: r['top_k'] = default_top_k
    if 'sample_size' not in r: r['sample_size'] = default_sample_size
    if 'note' not in r: r['note'] = default_note
    new_runs.append(r)
runs = new_runs

In [4]:
len(runs)

5

In [5]:
def generate_outputs(run, 
                     device=None, 
                     reward_device=None, 
                     generator_device=None,
                     evaluator_device=None,
                     raw_save_path='./raw',
                     summary_save_path='./summary'): 
    if device is not None: 
        reward_device=device
        generator_device=device
        evaluator_device=device
        
    model = run['model']
    task = run['task']
    sample_size = run['sample_size']
    top_k = run['top_k']
    
    dummy_prompts = {'pos2neg': '', 'neg2pos': ''}
    generator = PromptedGPT2Generator(model, 
                                      dummy_prompts,
                                      reward_device=reward_device,
                                      generator_device=generator_device)
    evaluator = Evaluator(evaluator_device)
    
    start = time.time()
    output_list = generator.sample_generate(task, 
                                            sample_size, 
                                            top_k=top_k, 
                                            top_p=None, 
                                            single_prompt=run['prompt'])
    time_elapsed = time.time() - start
    del generator
    
    summary, output_df = evaluator.evaluate_output(task, 
                                                   output_list)
    timestamp = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
    del evaluator
    
    summary.update({'model': model,
                    'task': task,
                    'sample_size': sample_size,
                    'top_k': top_k,
                    'lr': run['lr'],
                    'seed': run['seed'],
                    'machine': run['machine'],
                    'data': run['data'],
                    'prompt': run['prompt'],
                    'time_elapsed': round(time_elapsed, 2),
                    'timestamp': timestamp,
                    'note': run['note']})
    print(summary)
    
    output_name = f"{task}_{model}_{run['data']}_{run['machine']}_{timestamp}"
    json.dump(summary, open(os.path.join(summary_save_path, output_name + '.json'), 'w'))
    output_df.to_csv(os.path.join(raw_save_path, output_name + '.csv'), index=False)
    
    return summary, output_df

In [6]:
summaries, output_dfs = [], []
for _ in range(1): 
    for r in runs: 
        summary, output_df = generate_outputs(r, 
                                              reward_device=2, 
                                              generator_device=2, 
                                              evaluator_device=2)
        summaries.append(summary)
        output_dfs.append(output_df)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  2%|▏         | 10/500 [00:13<11:06,  1.36s/it]/opt/conda/envs/sql-203/lib/python3.8/site-packages/transformers/pipelines/base.py:997: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 500/

Comparing with reference...


100%|██████████| 500/500 [00:00<00:00, 5180.20it/s]


Running test classifier


100%|██████████| 500/500 [00:03<00:00, 133.99it/s]


...

Running test classifier


100%|██████████| 500/500 [00:03<00:00, 133.97it/s]


Computing perplexity...


100%|██████████| 500/500 [00:04<00:00, 103.90it/s]


{'sum_reward': 86.15, 'recon': 73.31, 'self_bleu': 42.93, 'ref_bleu': 23.44, 'style_acc': 95.8, 'ppl': 33.2, 'model': 'gpt2-medium', 'task': 'pos2neg', 'sample_size': 32, 'top_k': 10, 'lr': 0.0001, 'seed': 3, 'machine': 'ec2-239', 'data': 'all-train', 'prompt': 'Damage exceptionFailure Meaning Meaning', 'time_elapsed': 400.69, 'timestamp': '2022-05-09_14:36:21', 'note': ''}


In [7]:
summaries, output_dfs = [], []
for r in runs: 
    summary, output_df = generate_outputs(r, 
                                          reward_device=3, 
                                          generator_device=2, 
                                          evaluator_device=3)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
10it [00:04,  2.18it/s]/opt/conda/envs/sql-203/lib/python3.8/site-packages/transformers/pipelines/base.py:997: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
500it [03:48,  2.19it/s]


Comparing with reference...


100%|██████████| 500/500 [00:00<00:00, 5388.26it/s]


Running test classifier


100%|██████████| 500/500 [00:03<00:00, 128.80it/s]


Computing perplexity...


100%|██████████| 500/500 [00:04<00:00, 100.86it/s]


{'sum_reward': 75.34, 'recon': 52.16, 'self_bleu': 22.89, 'ref_bleu': 14, 'style_acc': 0.99, 'ppl': 34.68, 'model': 'distilgpt2', 'task': 'neg2pos', 'sample_size': 32, 'top_k': 10, 'lr': 0.0001, 'seed': 2, 'machine': 'petuum-42', 'data': '500-test', 'time_elapsed': 228.76, 'timestamp': '2022-04-25_22:24:27', 'note': ''}


In [32]:
from tqdm import tqdm
neg2pos_ref_output = []
for i, (src, ref) in tqdm(enumerate(zip(generator.sentence_dict['src_neg2pos'],
                                        evaluator.sentence_dict['ref_neg2pos']))): 
    output = generator._select_output([ref],
                                       [src], 
                                       'LABEL_1', 
                                       sample_id=i)
    neg2pos_ref_output.append(output)

500it [00:13, 38.04it/s]
